In [1]:
# ===================================================
# 0) LIBRARIES
# ===================================================
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score

# ===================================================
# 1) LOAD DATA
# ===================================================
computeddata = pd.read_excel("CleanedData.xlsx", sheet_name="Computed variables", nrows=54)
print("\n ComputedData Columns:", computeddata.columns.tolist())

id_col = "Participant #"

# ===================================================
# 2) COLUMN 
# ===================================================
demo_columns = [
    'Age','Level','SportType',
    'NumberOfAthletes','AthletesAges','Supervisor',
    'YrsExperience','YrsAtJob','ContractType', 
]

sums_columns = [
    'Role_clarity','Work_Life_Balance','Reward_Esteem_Sum',
    'Reward_Promotion_Sum','Reward_JobSecurity_Sum',
    'Orgz_Commitment_Sum','Relationships_Coworkers_Sum',
    'Relationships_Supervisors_Sum','Relationships_OrgCulture_Sum',
    'OrgSupport_Sum','WorkEngagment_Sum','OrgzFinances', 'HrsCoaching','HrsAdmintrative',
]

#removed_columns = 'AthletePerformance',

#sums_columns = [
#    'Exhaustion_Sum','Cynicism_Sum','ReducedProffesionalEfficacy_Sum',
#    'Turnover_Intetntions','ReducedJob_Satisfaction',
#    'Stress',
#]


all_feature_columns = list(dict.fromkeys(demo_columns + sums_columns))


selected_columns = all_feature_columns
print(f"\n Using {len(selected_columns)} columns for clustering")







 ComputedData Columns: ['Participant #', 'Gender', 'Age', 'Ethnicity', 'EducationLevel', 'MaritalStatus', 'Level', 'SportType', 'GenderOfAthletes', 'NumberOfAthletes', 'SpecificSports', 'AthletesAges', 'Supervisor', 'YrsExperience', 'YrsAtJob', 'ContractType', 'HrsCoaching', 'HrsAdmintrative', 'HrsFluctuation', 'ComutePerWeek', 'SickDaysTaken', 'SickDaysWanted', 'AthletePerformance', 'OrgzFinances', 'Role_clarity', 'Work_Life_Balance', 'Reward_Sum', 'Reward_Esteem_Sum', 'Reward_Promotion_Sum', 'Reward_JobSecurity_Sum', 'Orgz_Commitment_Sum', 'Relationships_Sum', 'Relationships_Coworkers_Sum', 'Relationships_Supervisors_Sum', 'Relationships_OrgCulture_Sum', 'OrgSupport_Sum', 'WorkEngagment_Sum', 'WorkAddiction_Sum', 'Exhaustion_Sum', 'Cynicism_Sum', 'ReducedProffesionalEfficacy_Sum', 'Turnover_Intetntions', 'ReducedJob_Satisfaction', 'Stress', 'Standerdised_IllBeing_Score (Exhaustion, Cynicsm, Turnover intentions, Reduced Job satisfaction, Stress)']

 Using 23 columns for clustering


In [2]:
# ===================================================
# 3) Group Level + AthletesAges 
# ===================================================
def map_level_group(level_str):
    if pd.isna(level_str): return np.nan
    tokens = [t.strip().lower() for t in str(level_str).replace(";", ",").split(",") if t.strip()]
    nat_alias = {"national competitions","professional leagues or tours"}
    intl_alias = {"international competitions"}
    other_alias = {"other","other (please specify)","others"}
    has_national = any(t in nat_alias for t in tokens)
    has_international = any(t in intl_alias for t in tokens)
    has_other = any(t in other_alias for t in tokens)
    if has_national and has_international: return "Both"
    if has_national: return "National"
    if has_international: return "International"
    if has_other: return "Other"
    return np.nan

computeddata["Level"] = computeddata["Level"].apply(map_level_group)

def map_athlete_age(val):
    if pd.isna(val) or str(val).strip()=="":
        return np.nan
    v = str(val).strip().lower()
    if "senior" in v: return "Senior"
    if "under 18" in v or "under 14" in v: return "Junior"
    return np.nan

computeddata["AthletesAges"] = computeddata["AthletesAges"].apply(map_athlete_age)


In [3]:
# ---- Convert hour ranges into ordered categories (1–7) ----
def map_hours_to_order(val):
    if pd.isna(val):
        return np.nan
    val = str(val).strip().lower()
    val = val.replace("–", "-").replace("—", "-")

    mapping = {
        "less than 6 hours/week": 1,
        "7 - 12 hours/week": 2,
        "13 - 18 hours/week": 3,
        "19 - 24 hours/week": 4,
        "25 - 30 hours/week": 5,
        "31 - 36 hours/week": 6,
        "37 or more hours/week": 7,
        
        "less than 1 hours/week": 1,   # for admin
        "1 - 4 hours/week": 2,
        "5 - 8 hours/week": 3,
        "9 - 12 hours/week": 4,
        "13 - 16 hours/week": 5,
        "17 - 20 hours/week": 6,
        "21 or more hours/week": 7,
    }

    return mapping.get(val, np.nan)


computeddata["HrsCoaching"] = computeddata["HrsCoaching"].apply(map_hours_to_order)
computeddata["HrsAdmintrative"] = computeddata["HrsAdmintrative"].apply(map_hours_to_order)

computeddata["TotalHours"] = computeddata["HrsCoaching"] + computeddata["HrsAdmintrative"]

selected_columns = list(dict.fromkeys(selected_columns + ["TotalHours"]))


In [4]:

# ===================================================
# 4) BUILD X 
# ===================================================

X_all = computeddata[[id_col] + selected_columns].copy()
X_all = X_all.dropna(subset=selected_columns)  


ids = X_all[id_col].copy()
X = X_all[selected_columns].copy()

print("after dropping NaN rows (features):", X.shape)
print("Has TotalHours in X?", "TotalHours" in X.columns)

# ===================================================
# 5) EXPLICIT ORDINAL ORDERS + MAPPING 
# ===================================================
def canon(s: str) -> str:
    if pd.isna(s): return ""
    s = str(s).strip().lower()
    s = s.replace("–","-").replace("—","-")
    s = re.sub(r"\s*-\s*","-", s)
    s = re.sub(r"\s+"," ", s)
    s = s.replace(" hours/week","").replace(" hours","")
    s = s.replace(" years","").replace(" year","")
    s = s.replace(" athletes","")
    s = s.replace(" or more","+").replace("over ",">")
    s = s.replace("less than ","<")
    return s

def map_with_order(series: pd.Series, ordered_labels: list[str]) -> pd.Series:
    mapping = {canon(lbl): i for i, lbl in enumerate(ordered_labels)}
    out = series.map(lambda v: mapping.get(canon(v), np.nan)).astype(float)
    unmapped = sorted({canon(v) for v in series.dropna().unique() if canon(v) not in mapping})
    if unmapped:
        print(f"⚠️ Unmapped values in {series.name}: {unmapped}")
    return out


age_order = ["18-24","25-34","35-44","45-54","55-64","65+"]
num_ath_order = ["1-5 athletes","6-10 athletes","11-15 athletes","16-20 athletes","21-30 athletes","30+"]
ath_ages_order = ["Junior","Senior"]
exp_order = ["Less than 1 year","1–2 years","2–5 years","5–10 years","10–20 years","20-30 years","Over 30 years"]
job_order = ["Less than 1 year","1–2 years","2–5 years","5–10 years","10–20 years","20-30 years","Over 30 years"]
#hrs_coach_order = ["Less than 6 hours/week","7 – 12 hours/week","13 – 18 hours/week","19 – 24 hours/week",
 #                  "25 – 30 hours/week","31 – 36 hours/week","37 or more hours/week"]
#hrs_admin_order = ["Less than 1 hours/week","1 – 4 hours/week","5 – 8 hours/week","9 – 12 hours/week",
 #                  "13 – 16 hours/week","17 – 20 hours/week","21 or more hours/week"]

explicit_orders = {
    "Age": age_order,
    "NumberOfAthletes": num_ath_order,
    "AthletesAges": ath_ages_order,
    "YrsExperience": exp_order,
    "YrsAtJob": job_order,
    #"HrsCoaching": hrs_coach_order,
    #"HrsAdmintrative": hrs_admin_order
}

categorical_nominal = ["Level","SportType","Supervisor","ContractType"]
categorical_ordinal = list(explicit_orders.keys())
numeric_cols = sums_columns 

after dropping NaN rows (features): (39, 24)
Has TotalHours in X? True


In [5]:
# ===================================================
# K-MEDOIDS (PAM) with Gower distance
# ===================================================
import numpy as np
import gower
from sklearn.metrics import silhouette_score

X_gower = X.copy()

for col, order in explicit_orders.items():
    X_gower[col] = map_with_order(X_gower[col], order)

for c in categorical_nominal:
    X_gower[c] = X_gower[c].astype(str)
for c in sums_columns:
    X_gower[c] = pd.to_numeric(X_gower[c], errors="coerce")

cat_mask = [col in categorical_nominal for col in X_gower.columns]
D = gower.gower_matrix(X_gower, cat_features=cat_mask) 
n = D.shape[0]

# ---------- PAM (Partitioning Around Medoids) implementation ----------
def pam_kmedoids(D, k, n_init=20, random_state=42):
    rng = np.random.default_rng(random_state)

    def total_cost(medoids):
        return D[:, medoids].min(axis=1).sum()

    def assign_labels(medoids):
        return D[:, medoids].argmin(axis=1)

    best = {"cost": np.inf, "medoids": None, "labels": None}

    for seed in range(n_init):
        medoids = np.array(rng.choice(n, size=k, replace=False))
        current_cost = total_cost(medoids)

        improved = True
        while improved:
            improved = False
            labels = assign_labels(medoids)
            non_medoids = np.setdiff1d(np.arange(n), medoids, assume_unique=True)
            for mi, m in enumerate(medoids):
                for h in non_medoids:
                    proposal = medoids.copy()
                    proposal[mi] = h
                    prop_cost = total_cost(proposal)
                    if prop_cost + 1e-12 < current_cost:
                        medoids = proposal
                        current_cost = prop_cost
                        improved = True
                        break
                if improved:
                    break

        labels = assign_labels(medoids)
        if current_cost < best["cost"]:
            best = {"cost": current_cost, "medoids": medoids.copy(), "labels": labels.copy()}

    return best["labels"], best["medoids"], best["cost"]

Ks = range(2, 7)
sil_kmed = []
labels_per_k = {}
medoids_per_k = {}

for k in Ks:
    labels_k, medoids_k, cost_k = pam_kmedoids(D, k, n_init=50, random_state=42)
    labels_per_k[k] = labels_k
    medoids_per_k[k] = medoids_k
    sil = silhouette_score(D, labels_k, metric="precomputed")
    sil_kmed.append(sil)
    print(f"[Gower+KMedoids] k={k}: silhouette={sil:.3f} | sizes={np.bincount(labels_k)} | cost={cost_k:.3f}")


chosen_k = 3  
labels_kmed = labels_per_k[chosen_k]
medoid_idx = medoids_per_k[chosen_k]

print(f"\n Using chosen k (Gower+KMedoids): {chosen_k}")
print("Cluster sizes (Gower+KMedoids):", np.bincount(labels_kmed))
print("Medoid row indices (relative to X):", medoid_idx)

# 5) Profiles 
X_with_clusters_kmed = X.copy()
X_with_clusters_kmed["Cluster"] = labels_kmed

print("\n Numeric scales (means by cluster) — Gower+KMedoids")
print(X_with_clusters_kmed.groupby("Cluster")[sums_columns].mean().round(2))

print("\n Demographic distributions by cluster — Gower+KMedoids")
for col in demo_columns:
    dist = (
        X_with_clusters_kmed.groupby("Cluster")[col]
        .value_counts(normalize=True).rename("proportion").mul(100).round(1)
        .reset_index()
        .pivot(index="Cluster", columns=col, values="proportion")
        .fillna(0.0).sort_index()
    )
    print(f"\n{col}")
    print(dist)

print("\n Medoid representatives (rows from your original data):")
print(computeddata.loc[X.index[medoid_idx], selected_columns])

[Gower+KMedoids] k=2: silhouette=0.097 | sizes=[23 16] | cost=8.753
[Gower+KMedoids] k=3: silhouette=0.130 | sizes=[14 19  6] | cost=8.146
[Gower+KMedoids] k=4: silhouette=0.136 | sizes=[14  6 17  2] | cost=7.688
[Gower+KMedoids] k=5: silhouette=0.119 | sizes=[11 16  4  2  6] | cost=7.237
[Gower+KMedoids] k=6: silhouette=0.101 | sizes=[ 6 12  3 10  6  2] | cost=6.862

 Using chosen k (Gower+KMedoids): 3
Cluster sizes (Gower+KMedoids): [14 19  6]
Medoid row indices (relative to X): [25  0 19]

 Numeric scales (means by cluster) — Gower+KMedoids
         Role_clarity  Work_Life_Balance  Reward_Esteem_Sum  \
Cluster                                                       
0                6.29               4.86               6.14   
1                5.16               3.47               5.95   
2                4.67               2.33               5.00   

         Reward_Promotion_Sum  Reward_JobSecurity_Sum  Orgz_Commitment_Sum  \
Cluster                                                 

In [6]:
# ===================================================
# Combined Cluster Profiles
# ===================================================

all_clusters = sorted(X_with_clusters_kmed["Cluster"].unique())

num_means = (
    X_with_clusters_kmed.groupby("Cluster")[sums_columns]
    .mean().round(2)
    .reindex(index=all_clusters)            
    .reset_index()
    .melt(id_vars="Cluster", var_name="Feature", value_name="Value")
)
num_means["Type"] = "Mean"

cat_props = []
for col in demo_columns:
    levels = sorted(map(str, X_with_clusters_kmed[col].dropna().unique()))
    dist = (
        X_with_clusters_kmed.groupby("Cluster")[col]
        .value_counts(normalize=True)
        .rename("Value").mul(100)
        .reset_index()
    )

    pivot = (
        dist.assign(**{col: dist[col].astype(str)})
            .pivot(index="Cluster", columns=col, values="Value")
            .reindex(index=all_clusters, columns=levels)    
            .fillna(0.0)                                    
            .round(1)
    )

    pivot = (
        pivot.reset_index()
             .melt(id_vars="Cluster", var_name=col, value_name="Value")
    )
    pivot["Feature"] = pivot[col].apply(lambda v: f"{col} | {v}")
    pivot["Type"] = "%"
    cat_props.append(pivot[["Cluster","Feature","Value","Type"]])

cat_props = pd.concat(cat_props, ignore_index=True)

profiles = pd.concat(
    [num_means[["Cluster","Feature","Value","Type"]],
     cat_props[["Cluster","Feature","Value","Type"]]],
    ignore_index=True
)

profile_table = (
    profiles.pivot_table(index=["Type","Feature"], columns="Cluster", values="Value", aggfunc="first")
            .reindex(columns=all_clusters)  
            .fillna(0.0)                     
)


profile_table = profile_table.sort_index(level=0, key=lambda x: x.map({"Mean": 0, "%": 1}))

with pd.option_context("display.max_rows", 300, "display.float_format", "{:.2f}".format):
    print(profile_table)

profile_table.to_excel("cluster_profiles_combined_kmedoids.xlsx")


Cluster                                                    0     1      2
Type Feature                                                             
Mean HrsAdmintrative                                    5.86  4.79   6.50
     HrsCoaching                                        4.71  3.53   5.83
     OrgSupport_Sum                                    32.79 29.37  19.83
     OrgzFinances                                       5.36  5.42   3.17
     Orgz_Commitment_Sum                               28.79 27.53  22.17
     Relationships_Coworkers_Sum                       13.86 10.95   6.83
     Relationships_OrgCulture_Sum                      15.93 13.89  10.67
     Relationships_Supervisors_Sum                     18.21 17.79  13.83
     Reward_Esteem_Sum                                  6.14  5.95   5.00
     Reward_JobSecurity_Sum                             4.50  5.32   3.67
     Reward_Promotion_Sum                               7.71  7.58   6.33
     Role_clarity                     

In [7]:
participants_clusters_kmed = (
    computeddata.loc[X.index, ["Participant #"]]
    .assign(Cluster=labels_kmed)     
)

by_cluster = (
    participants_clusters_kmed.groupby("Cluster")["Participant #"]
    .apply(list)
)
for c, ids in by_cluster.items():
    print(f"Cluster {c}: {ids}")

Cluster 0: [5, 8, 12, 16, 24, 27, 31, 32, 35, 41, 45, 46, 49, 50]
Cluster 1: [1, 2, 3, 4, 9, 14, 17, 19, 20, 21, 23, 25, 28, 30, 36, 37, 38, 44, 51]
Cluster 2: [13, 22, 26, 29, 47, 53]
